<a href="https://colab.research.google.com/github/tushant-akar/CS367-Artifical-Intelligence-Lab/blob/main/MENACE_(Lab_Assignment_7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
from tqdm import tqdm

class MENACE:
    def __init__(self, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1, exploration_rate_decay=0.9995):
        self.matchboxes = {}
        self.moves_made = []
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_rate_decay = exploration_rate_decay

    def get_move(self, board):
        if board not in self.matchboxes:
            self.matchboxes[board] = {i: 0 for i in range(9) if board[i] == '-'}

        moves = self.matchboxes[board]
        if random.random() < self.exploration_rate:
            move = random.choice(list(moves.keys()))
        else:
            move = max(moves, key=moves.get)

        self.moves_made.append((board, move))
        return move

    def learn(self, final_reward):
        for i in range(len(self.moves_made) - 1, -1, -1):
            board, move = self.moves_made[i]
            next_board = self.moves_made[i+1][0] if i+1 < len(self.moves_made) else None

            if next_board:
                next_max_q = max(self.matchboxes[next_board].values()) if self.matchboxes[next_board] else 0
                reward = self.calculate_reward(board, move)
                new_q = (1 - self.learning_rate) * self.matchboxes[board][move] + \
                        self.learning_rate * (reward + self.discount_factor * next_max_q)
            else:
                new_q = final_reward

            self.matchboxes[board][move] = new_q

        self.moves_made = []
        self.exploration_rate *= self.exploration_rate_decay  # Decay the exploration rate

    def calculate_reward(self, board, move):
        new_board = board[:move] + 'X' + board[move+1:]
        if check_win(new_board):
            return 1  # Immediate win
        elif check_win(new_board.replace('X', 'O').replace('-', 'X')):
            return 0.5  # Blocked opponent's win
        else:
            return 0.1  # Neither win nor block

def check_win(board):
    winning_combinations = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
        (0, 4, 8), (2, 4, 6)  # Diagonals
    ]
    for a, b, c in winning_combinations:
        if board[a] == board[b] == board[c] != '-':
            return True
    return False

class MinimaxPlayer:
    def __init__(self, max_depth=5):
        self.max_depth = max_depth

    def get_move(self, board):
        best_move = None
        best_score = -math.inf

        for move in range(9):
            if board[move] == '-':
                new_board = board[:move] + 'O' + board[move+1:]
                score = self.minimax(new_board, 0, False)
                if score > best_score:
                    best_score = score
                    best_move = move

        return best_move

    def minimax(self, board, depth, is_maximizing):
        if check_win(board):
            return -1 if is_maximizing else 1
        elif '-' not in board:
            return 0

        if is_maximizing:
            best_score = -math.inf
            for move in range(9):
                if board[move] == '-':
                    new_board = board[:move] + 'O' + board[move+1:]
                    score = self.minimax(new_board, depth + 1, False)
                    best_score = max(best_score, score)
            return best_score
        else:
            best_score = math.inf
            for move in range(9):
                if board[move] == '-':
                    new_board = board[:move] + 'X' + board[move+1:]
                    score = self.minimax(new_board, depth + 1, True)
                    best_score = min(best_score, score)
            return best_score

def play_game(menace, opponent):
    board = '-' * 9
    current_player = menace

    while True:
        move = current_player.get_move(board)
        board = board[:move] + ('X' if current_player == menace else 'O') + board[move+1:]

        if check_win(board):
            return 1 if current_player == menace else -1
        if '-' not in board:
            return 0

        current_player = opponent if current_player == menace else menace

def train_menace(menace, opponent, episodes):
    for episode in tqdm(range(episodes), desc="Training MENACE"):
        result = play_game(menace, opponent)
        menace.learn(result)

# Training
menace = MENACE()
opponent = MinimaxPlayer(max_depth=5)

print("Training MENACE...")
train_menace(menace, opponent, 20000)

print("\nTraining completed!")

# Test game
board = '-' * 9
current_player = menace

print("\nLet's play a game against MENACE!")
while True:
    print(f"\nCurrent board:\n{board[:3]}\n{board[3:6]}\n{board[6:]}")

    if current_player == menace:
        move = menace.get_move(board)
        print(f"MENACE plays: {move}")
    else:
        move = int(input("Your move (0-8): "))
        while board[move] != '-':
            move = int(input("Invalid move. Try again (0-8): "))

    board = board[:move] + ('X' if current_player == menace else 'O') + board[move+1:]

    if check_win(board):
        print(f"\nFinal board:\n{board[:3]}\n{board[3:6]}\n{board[6:]}")
        print("MENACE wins!" if current_player == menace else "You win!")
        break
    if '-' not in board:
        print(f"\nFinal board:\n{board[:3]}\n{board[3:6]}\n{board[6:]}")
        print("It's a draw!")
        break

    current_player = opponent if current_player == menace else menace

# Final learning step
if current_player == menace:
    menace.learn(-1)  # MENACE lost
else:
    menace.learn(1)  # MENACE won

Training MENACE...

Current board:
O--
-X-
---
Exploiting: Chose move 1 with Q-value 1.35

Current board:
OX-
-X-
---

Current board:
OX-
-X-
-O-
Exploiting: Chose move 3 with Q-value 0.95

Current board:
OX-
XX-
-O-

Current board:
OX-
XXO
-O-
Exploring: Chose move 2

Current board:
OXX
XXO
-O-

Current board:
OXX
XXO
OO-
Exploring: Chose move 8
Updated Q-value for move 8 on board OXXXXOOO-: 0.00
Updated Q-value for move 2 on board OX-XXO-O-: 0.50
Updated Q-value for move 3 on board OX--X--O-: 0.95
Updated Q-value for move 1 on board O---X----: 1.35
Updated Q-value for move 4 on board ---------: 1.72
Exploring: Chose move 2


Training MENACE:   0%|          | 1/20000 [00:00<1:22:43,  4.03it/s]

Exploiting: Chose move 0 with Q-value 1.24
Exploiting: Chose move 7 with Q-value 0.90
Exploiting: Chose move 5 with Q-value 0.49
Exploiting: Chose move 6 with Q-value 0.00
Updated Q-value for move 6 on board XOXOOX-XO: 0.00
Updated Q-value for move 5 on board XOXOO--X-: 0.49
Updated Q-value for move 7 on board XOX-O----: 0.91
Updated Q-value for move 0 on board --X-O----: 1.24
Updated Q-value for move 2 on board ---------: 1.53
Exploiting: Chose move 4 with Q-value 1.72


Training MENACE:   0%|          | 2/20000 [00:00<1:28:45,  3.76it/s]

Exploiting: Chose move 1 with Q-value 1.35
Exploiting: Chose move 3 with Q-value 0.95
Exploiting: Chose move 2 with Q-value 0.50
Exploiting: Chose move 8 with Q-value 0.00
Updated Q-value for move 8 on board OXXXXOOO-: 0.00
Updated Q-value for move 2 on board OX-XXO-O-: 0.50
Updated Q-value for move 3 on board OX--X--O-: 0.95
Updated Q-value for move 1 on board O---X----: 1.35
Updated Q-value for move 4 on board ---------: 1.72
Exploiting: Chose move 4 with Q-value 1.72


...

Exploiting: Chose move 0 with Q-value 1.32
Exploiting: Chose move 1 with Q-value 0.94
Exploiting: Chose move 6 with Q-value 0.50
Exploiting: Chose move 8 with Q-value 0.00
Updated Q-value for move 8 on board XXOOOXXO-: 0.00
Updated Q-value for move 6 on board XXOOOX---: 0.50
Updated Q-value for move 1 on board X-OO-X---: 0.94
Updated Q-value for move 0 on board --O--X---: 1.33
Updated Q-value for move 5 on board ---------: 1.65
Exploring: Chose move 4
Exploiting: Chose move 1 with Q-value 1.35
Exploiting: Chose move 3 with Q-value 0.95
Exploiting: Chose move 2 with Q-value 0.50
Exploiting: Chose move 8 with Q-value 0.00
Updated Q-value for move 8 on board OXXXXOOO-: 0.00
Updated Q-value for move 2 on board OX-XXO-O-: 0.50
Updated Q-value for move 3 on board OX--X--O-: 0.95
Updated Q-value for move 1 on board O---X----: 1.35
Updated Q-value for move 4 on board ---------: 1.72
Exploiting: Chose move 0 with Q-value 1.72


Training MENACE:  13%|█▎        | 2596/20000 [15:14<1:42:37,  2.83it/s]

Exploiting: Chose move 5 with Q-value 1.36
Exploiting: Chose move 7 with Q-value 0.95
Exploiting: Chose move 2 with Q-value 0.50
Exploiting: Chose move 3 with Q-value 0.00
Updated Q-value for move 3 on board XOX-OXOXO: 0.00
Updated Q-value for move 2 on board XO--OXOX-: 0.50
Updated Q-value for move 7 on board XO--OX---: 0.95
Updated Q-value for move 5 on board X---O----: 1.36
Updated Q-value for move 0 on board ---------: 1.72
Exploiting: Chose move 0 with Q-value 1.72


Training MENACE:  13%|█▎        | 2598/20000 [15:15<1:38:45,  2.94it/s]

Exploiting: Chose move 5 with Q-value 1.36
Exploiting: Chose move 7 with Q-value 0.95
Exploiting: Chose move 2 with Q-value 0.50
Exploiting: Chose move 3 with Q-value 0.00
Updated Q-value for move 3 on board XOX-OXOXO: 0.00
Updated Q-value for move 2 on board XO--OXOX-: 0.50
Updated Q-value for move 7 on board XO--OX---: 0.95
Updated Q-value for move 5 on board X---O----: 1.36
Updated Q-value for move 0 on board ---------: 1.72
Exploiting: Chose move 0 with Q-value 1.72
